In [17]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [18]:
# Query 3: Zeigt, wie viele Events jeder Arzt betreut
# hilfreich zur Einschätzung der Arbeitsbelastung.

query = """
SELECT
    a.vorname || ' ' || a.nachname AS arzt_name,
    COUNT(ae.event_id) AS anzahl_events
FROM arzt a
LEFT JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
GROUP BY a.arzt_id
ORDER BY anzahl_events DESC;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)



['arzt_name', 'anzahl_events']
('Sascha Bader', 72)
('Gabriele Marti', 54)
('Susi Lehmann', 50)
('Mohammed Bär', 41)
('Annick Graf', 33)
('Norman Moser', 23)
('Karin Vogel', 18)
('Erik Brunner', 18)


In [19]:
# Query 1: Zeigt alle aktuell offenen (nicht abgeschlossenen) Behandlungen
# inklusive Patient, Station und Startdatum.

query = """
SELECT
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station,
    b.beschreibung,
    b.start_datum
FROM behandlung b
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
WHERE b.abgeschlossen = false
ORDER BY b.start_datum;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['patient_name', 'station', 'beschreibung', 'start_datum']
('Antoinette Hirt', 'Chirurgie', 'Präoperative Abklärung', datetime.date(2025, 12, 14))
('Meryem Herzog', 'Radiologie', 'Herz-Kreislauf Check', datetime.date(2025, 12, 14))
('Ginette Rey', 'Innere Medizin', 'Herz-Kreislauf Check', datetime.date(2025, 12, 14))
('Serge Hofer', 'Pädiatrie', 'Magen-Darm Untersuchung', datetime.date(2025, 12, 15))
('Rita Ammann', 'Pädiatrie', 'Herz-Kreislauf Check', datetime.date(2025, 12, 18))
('Rosalie Schärer', 'Innere Medizin', 'Blutdruckeinstellung', datetime.date(2025, 12, 18))
('Victoria Schnyder', 'Chirurgie', 'Magen-Darm Untersuchung', datetime.date(2025, 12, 21))
('Marcela Beck', 'Radiologie', 'Blutdruckeinstellung', datetime.date(2025, 12, 22))
('Enrico Favre', 'Innere Medizin', 'Präoperative Abklärung', datetime.date(2025, 12, 25))
('Daniele Wehrli', 'Radiologie', 'Schmerztherapie', datetime.date(2025, 12, 26))
('Lotte Maier', 'Radiologie', 'Appendektomie Nachkontrolle', datetime.date(20

In [20]:
# Query 2: Zeigt pro Station, wie viele Betten aktuell frei sind.

query = """
SELECT
    s.name AS station,
    COUNT(b.bett_id) AS freie_betten
FROM bett b
JOIN station s ON b.station_id = s.station_id
WHERE b.status = 'frei'
GROUP BY s.name
ORDER BY freie_betten DESC;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['station', 'freie_betten']
('Chirurgie', 2)
('Innere Medizin', 2)
('Intensivstation', 2)
('Notfall', 2)
('Pädiatrie', 2)
('Radiologie', 2)


In [21]:
# Query 4: Zeigt alle Patienten mit den Medikamenten,
# die ihnen im Rahmen von Events verabreicht wurden.

query = """
SELECT
    p.vorname || ' ' || p.nachname AS patient_name,
    m.name AS medikament,
    m.wirkstoff
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
JOIN event e ON b.behandlungs_id = e.behandlung_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
ORDER BY patient_name;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['patient_name', 'medikament', 'wirkstoff']
('Adrian Schärer', 'Ibuprofen', 'Ibuprofen')
('Adrian Schärer', 'Amoxicillin', 'Amoxicillin')
('Adrian Schärer', 'Aspirin', 'Acetylsalicylsäure')
('Adrian Schärer', 'Ibuprofen', 'Ibuprofen')
('Afrim Gerber', 'Paracetamol', 'Paracetamol')
('Afrim Gerber', 'Amoxicillin', 'Amoxicillin')
('Afrim Gerber', 'Amoxicillin', 'Amoxicillin')
('Afrim Gerber', 'Insulin', 'Insulin')
('Agnes Grob', 'Amoxicillin', 'Amoxicillin')
('Agnes Grob', 'Aspirin', 'Acetylsalicylsäure')
('Agnes Grob', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Insulin', 'Insulin')
('Alfredo Fischer', 'Insulin', 'Insulin')
('Alfredo Fischer', 'Aspirin', 'Acetylsalicylsäure')
('Alyssa Stauffer', 'Ibuprofen', 'Ibuprofen')
('Alyssa Stauffer', 'Insulin', 'Insulin')
('Alyssa Stauffer', 'Paracetamol', 'Paracetamol')
('Alyssa Stauffer', 'Amoxicillin', 'Amoxicillin')
('Alyssa Stauffer', 'Insulin', 'Insul

In [22]:
# Query 5: Zeigt alle Events des heutigen Tages,
# sortiert nach Raum und Uhrzeit (z. B. für Tagesplanung).

query = """
SELECT
    r.art AS raum_art,
    r.raumnummer,
    e.uhrzeit,
    p.vorname || ' ' || p.nachname AS patient_name
FROM event e
JOIN raum r ON e.raumid = r.raumnummer
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE e.datum = CURRENT_DATE
ORDER BY r.raumnummer, e.uhrzeit;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['raum_art', 'raumnummer', 'uhrzeit', 'patient_name']
('Operationssaal', 1, datetime.time(0, 54, 29), 'Kathleen Portmann')
('Operationssaal', 1, datetime.time(0, 59, 4), 'Pierina Kaufmann')
('Operationssaal', 1, datetime.time(6, 28, 45), 'Karine Ferrari')
('Operationssaal', 1, datetime.time(14, 44, 25), 'Jeannette Maurer')
('Operationssaal', 1, datetime.time(15, 28, 47), 'Afrim Gerber')
('Operationssaal', 1, datetime.time(17, 8, 6), 'Elena Bolliger')
('Untersuchungsraum', 2, datetime.time(5, 1, 32), 'Ilaria Wegmann')
('Untersuchungsraum', 2, datetime.time(7, 48, 21), 'Rosalie Schärer')
('Untersuchungsraum', 2, datetime.time(10, 33, 30), 'Cristina Sutter')
('Untersuchungsraum', 2, datetime.time(15, 35, 5), 'Albina Schmidt')
('Untersuchungsraum', 2, datetime.time(15, 47, 24), 'Nebojsa Schär')
('Untersuchungsraum', 2, datetime.time(23, 1, 13), 'Raymond Meister')
('Operationssaal', 3, datetime.time(0, 29, 31), 'Emil Steinmann')
('Operationssaal', 3, datetime.time(5, 32, 44), 'Ryan Rüegg')
